# Field-Mappable Tuple List Example

In [1]:
import fmtl
from fmtl.contrib import * #Holds all helper functions

Let's create some data: FMTL works on tuple lists.

In [2]:
data = [("a", 1, ["a","b"],[[["a",["a","b"]],"b"]]), ("b", 2, ["b","a"],[[["a","b",["b"]],"b"]])]
rows = ("letter","num","seq","nested")

## Basics

`FMTL(tuplelist,rows=None,checks=None)` rows and checks arguments are mandatory:
 - rows is for friendly field adressing
 - checks, if provided, is called at init on each tuple.

In [3]:
tl = fmtl.FMTL(data,rows)
print("num is field #"+str(tl.f2i("num")))

num is field #1


Fields can be mapped using `tl.set_mapping(field,map_func,unk=None)` using a dictionnary or a function.

In [4]:
tl.set_mapping("letter",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))
print()
tl.set_mapping("num",lambda x:x+10)
print("==> " + str(list(tl)))

==> [('Ah', 1, ['a', 'b'], [[['a', ['a', 'b']], 'b']]), ('Bi', 2, ['b', 'a'], [[['a', 'b', ['b']], 'b']])]

==> [('Ah', 11, ['a', 'b'], [[['a', ['a', 'b']], 'b']]), ('Bi', 12, ['b', 'a'], [[['a', 'b', ['b']], 'b']])]


Dictionnary based mappings are automatically applied to sequences (nested or not).

In [5]:
tl.set_mapping("seq",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))
print()
tl.set_mapping("nested",{"a":"Ah","b":"Bi"})
print("==> " + str(list(tl)))

==> [('Ah', 11, ['Ah', 'Bi'], [[['a', ['a', 'b']], 'b']]), ('Bi', 12, ['Bi', 'Ah'], [[['a', 'b', ['b']], 'b']])]

==> [('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']]), ('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])]


Mapping is done as iteration time in the object's `__get_item__()` method.

In [16]:
print(str(tl.tuplelist[0])+" --maps to--> "+str(tl[0]))

('a', 1, ['a', 'b'], [[['a', ['a', 'b']], 'b']]) --maps to--> ('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']])


You still can prebuild it explicitely, it's just a list after all.

In [20]:
builded = list(tl) #if you don't have too much data, else use a generator
print(builded)

[('Ah', 11, ['Ah', 'Bi'], [[['Ah', ['Ah', 'Bi']], 'Bi']]), ('Bi', 12, ['Bi', 'Ah'], [[['Ah', 'Bi', ['Bi']], 'Bi']])]


## Useful Helpers

In [ ]:
#todo: see contrib.py for existing function details:
# -> indexed_iter(fmtl,idxs)
# -> get_stats(fmtl, field, key_iter=None, verbose=False)
# -> get_field_dict(fmtl, field, offset=0, max_count=-1, key_iter=None, iter_func=None)